## ------------------------------------------
## SCRIPT DE TRAITEMENT DE LA BASE DE DONNÉES KAGGLE (CleansingWine.csv)
### Objectif : nettoyer, compléter et structurer les données pour insertion dans Oracle
### Tables cibles : VINR, PRODUCTEURR, RECOLTER
## ------------------------------------------

### Installation des bibliothèques nécessaires :
#### pandas      → pour manipuler les données tabulaires (DataFrame)
#### numpy       → pour les opérations numériques (ex : NaN, conditions)
#### faker       → pour générer des prénoms aléatoires réalistes
#### googletrans → pour traduire automatiquement les noms de régions

In [51]:
pip install pandas numpy faker googletrans==4.0.0-rc1

Note: you may need to restart the kernel to use updated packages.


In [30]:
import pandas as pd
import numpy as np
from faker import Faker
from googletrans import Translator

### 1. Chargement et aperçu des données

In [31]:
df = pd.read_csv("cleansingWine.csv")
df.head(10)

/tmp/ipykernel_5154/1664116324.py:1: DtypeWarning: Columns (8,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("cleansingWine.csv")


,Unnamed: 0,id,name,producer,nation,local1,local2,local3,local4,varieties1,...,use,abv,degree,sweet,acidity,body,tannin,price,year,ml
0,0,137197,Altair,Altair,Chile,Rapel Valley,NaN,NaN,NaN,Cabernet Sauvignon,...,Table,14~15,17~19,SWEET1,ACIDITY4,BODY5,TANNIN4,220000,2014,750
1,1,137198,"Altair, Sideral",Altair,Chile,Rapel Valley,NaN,NaN,NaN,Cabernet Sauvignon,...,Table,14~15,16~18,SWEET1,ACIDITY3,BODY4,TANNIN4,110000,2016,750
2,2,137199,Baron du Val Red,Baron du Val,France,NaN,NaN,NaN,NaN,Carignan,...,Table,11~12,15~17,SWEET2,ACIDITY3,BODY2,TANNIN2,0,0,750
3,3,137200,Baron du Val White,Baron du Val,France,NaN,NaN,NaN,NaN,Carignan,...,Table,11~12,9~11,SWEET1,ACIDITY3,BODY2,TANNIN1,0,0,750
4,4,137201,"Benziger, Cabernet Sauvignon",Benziger,USA,California,NaN,NaN,NaN,Cabernet Sauvignon,...,Table,13~14,17~19,SWEET1,ACIDITY3,BODY3,TANNIN4,0,2003,750
5,5,137202,"Calvet, Beaujolais Villages",Calvet,France,Bourgogne,Beaujolais,NaN,NaN,Gamay,...,Table,12,13~15,SWEET2,ACIDITY4,BODY2,TANNIN2,0,2007,750
6,6,137203,"Calvet, Aetos",Calvet,France,Bordeaux,NaN,NaN,NaN,Merlot,...,Table,13,17~19,SWEET1,ACIDITY3,BODY4,TANNIN3,0,2003,750
7,7,137204,"Calvet, Bordeaux Red",Calvet,France,Bordeaux,NaN,NaN,NaN,Merlot,...,Table,13~14,16~18,SWEET1,ACIDITY3,BODY2,TANNIN3,21000,2011,750
8,8,137205,"Calvet, Bordeaux white",Calvet,France,Bordeaux,NaN,NaN,NaN,Sauvignon blanc,...,Table,11.5,9~11,SWEET1,ACIDITY4,BODY2,TANNIN1,21000,2010,750
9,9,137207,"Calvet, Varietals Cabernet Sauvignon",Calvet,France,Southern France,Languedoc Roussillon,NaN,NaN,Cabernet Sauvignon,...,Table,13~14,16~18,SWEET1,ACIDITY3,BODY2,TANNIN3,19000,2017,750


### 2. Suppression des colonnes inutiles

In [33]:
colonnes_a_supprimer = [
    'Unnamed: 0', 'nation', 'local2', 'local3', 'local4', 'varieties1', 'varieties2',
    'use', 'degree', 'sweet', 'acidity', 'tannin', 'price', 'body', 'varieties3',
    'varieties4', 'varieties5', 'varieties6', 'varieties7', 'varieties8', 'varieties9',
    'varieties10', 'varieties11', 'varieties12', 'type'
]
df.drop(columns=colonnes_a_supprimer, inplace=True)
df.head(20)

,id,name,producer,local1,abv,year,ml
0,137197,Altair,Altair,Rapel Valley,14~15,2014,750
1,137198,"Altair, Sideral",Altair,Rapel Valley,14~15,2016,750
2,137199,Baron du Val Red,Baron du Val,NaN,11~12,0,750
3,137200,Baron du Val White,Baron du Val,NaN,11~12,0,750
4,137201,"Benziger, Cabernet Sauvignon",Benziger,California,13~14,2003,750
5,137202,"Calvet, Beaujolais Villages",Calvet,Bourgogne,12,2007,750
6,137203,"Calvet, Aetos",Calvet,Bordeaux,13,2003,750
7,137204,"Calvet, Bordeaux Red",Calvet,Bordeaux,13~14,2011,750
8,137205,"Calvet, Bordeaux white",Calvet,Bordeaux,11.5,2010,750
9,137207,"Calvet, Varietals Cabernet Sauvignon",Calvet,Southern France,13~14,2017,750


### 3. Renommage des colonnes principales

In [34]:
df = df.rename(columns={'id': 'Nvin', 'name': 'Cru', 'producer': 'Nom', 'local1': 'Region', 'abv': 'Degre', 'year': 'Annee', 'ml': 'Qte'})
df.head(20)

,Nvin,Cru,Nom,Region,Degre,Annee,Qte
0,137197,Altair,Altair,Rapel Valley,14~15,2014,750
1,137198,"Altair, Sideral",Altair,Rapel Valley,14~15,2016,750
2,137199,Baron du Val Red,Baron du Val,NaN,11~12,0,750
3,137200,Baron du Val White,Baron du Val,NaN,11~12,0,750
4,137201,"Benziger, Cabernet Sauvignon",Benziger,California,13~14,2003,750
5,137202,"Calvet, Beaujolais Villages",Calvet,Bourgogne,12,2007,750
6,137203,"Calvet, Aetos",Calvet,Bordeaux,13,2003,750
7,137204,"Calvet, Bordeaux Red",Calvet,Bordeaux,13~14,2011,750
8,137205,"Calvet, Bordeaux white",Calvet,Bordeaux,11.5,2010,750
9,137207,"Calvet, Varietals Cabernet Sauvignon",Calvet,Southern France,13~14,2017,750


### 4. Vérification des valeurs manquantes

In [35]:
print(df.isnull().sum())

Nvin         0
Cru          0
Nom          0
Region     900
Degre     7146
Annee        0
Qte          0
dtype: int64


### 5. Nettoyage et normalisation de la colonne Degre (Degré d’alcool)
###    Format attendu : float entre 5 et 15

In [38]:
def convert_Degre(val): 
    try:
        val = str(val).strip()
        if "~" in val:
            a, b = map(float, val.split("~"))
            return round((a + b) / 2, 1)
        return float(val)
    except:
        return np.nan

df["Degre"] = df["Degre"].apply(convert_Degre)
df["Degre"].fillna(12.5, inplace=True)
df = df[(df["Degre"] >= 5) & (df["Degre"] <= 15)]
print(df["Degre"].head(15))

0     14.5
1     14.5
2     11.5
3     11.5
4     13.5
5     12.0
6     13.0
7     13.5
8     11.5
9     13.5
10    13.5
11    11.5
12    13.0
13    12.5
14    12.5
Name: Degre, dtype: float64


In [39]:
print(df.isnull().sum())

Nvin        0
Cru         0
Nom         0
Region    851
Degre       0
Annee       0
Qte         0
dtype: int64


### 6. Traitement des valeurs manquantes dans Region

In [40]:
regions = ["Rhone", "Beaujolais", "Sud Ouest", "Jura"] 
df["Region"] = df["Region"].apply(lambda x: x if pd.notnull(x) else np.random.choice(regions))
print(df["Region"].head(15))

0        Rapel Valley
1        Rapel Valley
2               Rhone
3               Rhone
4          California
5           Bourgogne
6            Bordeaux
7            Bordeaux
8            Bordeaux
9     Southern France
10    Southern France
11           Bordeaux
12           Bordeaux
13           Bordeaux
14           Bordeaux
Name: Region, dtype: object


In [41]:
print(df.isnull().sum())

Nvin      0
Cru       0
Nom       0
Region    0
Degre     0
Annee     0
Qte       0
dtype: int64


### 7. Génération aléatoire de prénoms et d’identifiants producteurs (Nprod)

In [43]:
fake = Faker('fr_FR')
prenoms = [fake.first_name() for _ in range(len(df))]
nprods = range(130, 130 + len(df))
df["Prenom"] = prenoms
df["Nprod"]  = nprods
df.head(20)

,Nvin,Cru,Nom,Region,Degre,Annee,Qte,Prenom,Nprod
0,137197,Altair,Altair,Rapel Valley,14.5,2014,750,Bertrand,130
1,137198,"Altair, Sideral",Altair,Rapel Valley,14.5,2016,750,Eugène,131
2,137199,Baron du Val Red,Baron du Val,Rhone,11.5,0,750,Marine,132
3,137200,Baron du Val White,Baron du Val,Rhone,11.5,0,750,Arthur,133
4,137201,"Benziger, Cabernet Sauvignon",Benziger,California,13.5,2003,750,Victoire,134
5,137202,"Calvet, Beaujolais Villages",Calvet,Bourgogne,12.0,2007,750,Marine,135
6,137203,"Calvet, Aetos",Calvet,Bordeaux,13.0,2003,750,Aurore,136
7,137204,"Calvet, Bordeaux Red",Calvet,Bordeaux,13.5,2011,750,Xavier,137
8,137205,"Calvet, Bordeaux white",Calvet,Bordeaux,11.5,2010,750,Marcelle,138
9,137207,"Calvet, Varietals Cabernet Sauvignon",Calvet,Southern France,13.5,2017,750,Manon,139


In [44]:
print(df.isnull().sum())

Nvin      0
Cru       0
Nom       0
Region    0
Degre     0
Annee     0
Qte       0
Prenom    0
Nprod     0
dtype: int64


### 8. Nettoyage de la colonne Annee : remplacer les 0 par une année valide

In [45]:
nb_zero = (df["Annee"] == 0).sum()
nb_zero

2389

In [46]:
annees = [2020, 2019, 2018, 2017, 2016, 2015]
df["Annee"] = df["Annee"].apply(lambda x: np.random.choice(annees) if x == 0 else x) 
df.head(20)

,Nvin,Cru,Nom,Region,Degre,Annee,Qte,Prenom,Nprod
0,137197,Altair,Altair,Rapel Valley,14.5,2014,750,Bertrand,130
1,137198,"Altair, Sideral",Altair,Rapel Valley,14.5,2016,750,Eugène,131
2,137199,Baron du Val Red,Baron du Val,Rhone,11.5,2015,750,Marine,132
3,137200,Baron du Val White,Baron du Val,Rhone,11.5,2017,750,Arthur,133
4,137201,"Benziger, Cabernet Sauvignon",Benziger,California,13.5,2003,750,Victoire,134
5,137202,"Calvet, Beaujolais Villages",Calvet,Bourgogne,12.0,2007,750,Marine,135
6,137203,"Calvet, Aetos",Calvet,Bordeaux,13.0,2003,750,Aurore,136
7,137204,"Calvet, Bordeaux Red",Calvet,Bordeaux,13.5,2011,750,Xavier,137
8,137205,"Calvet, Bordeaux white",Calvet,Bordeaux,11.5,2010,750,Marcelle,138
9,137207,"Calvet, Varietals Cabernet Sauvignon",Calvet,Southern France,13.5,2017,750,Manon,139


In [47]:
nb_zero = (df["Annee"] == 0).sum()
nb_zero

0

### 9. Traduction des noms de régions (anglais → français)

In [16]:
pip install googletrans==4.0.0-rc1

Note: you may need to restart the kernel to use updated packages.


In [48]:
translator = Translator()

unique_regions = df["Region"].unique()

translations = {}
for reg in unique_regions:
    try:
        res = translator.translate(reg, src="en", dest="fr")
        translations[reg] = res.text        # .text c'est le texte traduit
    except Exception as e:
        translations[reg] = reg

df["Region"] = df["Region"].map(translations)

### 10. Création des trois tables pour insertion dans Oracle

In [49]:
# Table VINR : identifiant vin, appellation, degré
df_vinr = df[["Nvin", "Cru", "Degre"]].drop_duplicates()
df_vinr.to_csv("vinR.csv", index=False)

# Table PRODUCTEURR : identifiant producteur, nom, prénom, région
df_producteurr = df[["Nprod", "Nom", "Prenom", "Region"]].drop_duplicates()
df_producteurr.to_csv("producteurR.csv", index=False)

# Table RECOLTER : relation producteur-vin avec année et quantité
df_recolter = df[["Nprod", "Nvin", "Annee", "Qte"]].drop_duplicates()
df_recolter.to_csv("recolteR.csv", index=False)

### 11. Vérification finale

In [50]:
print("Aperçu VINR :")
print(df_vinr.head(10))
print("\nAperçu PRODUCTEURR :")
print(df_producteurr.head(10))
print("\nAperçu RECOLTER :")
print(df_recolter.head(10))

Aperçu VINR :
     Nvin                                   Cru  Degre
0  137197                                Altair   14.5
1  137198                       Altair, Sideral   14.5
2  137199                      Baron du Val Red   11.5
3  137200                    Baron du Val White   11.5
4  137201          Benziger, Cabernet Sauvignon   13.5
5  137202           Calvet, Beaujolais Villages   12.0
6  137203                         Calvet, Aetos   13.0
7  137204                  Calvet, Bordeaux Red   13.5
8  137205                Calvet, Bordeaux white   11.5
9  137207  Calvet, Varietals Cabernet Sauvignon   13.5

Aperçu PRODUCTEURR :
   Nprod           Nom    Prenom            Region
0    130        Altair  Bertrand     Vallée du vif
1    131        Altair    Eugène     Vallée du vif
2    132  Baron du Val    Marine             Rhône
3    133  Baron du Val    Arthur             Rhône
4    134      Benziger  Victoire        Californie
5    135        Calvet    Marine         Bourgogne
6 